## Softmax Classifier

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.set_random_seed(777)  # for reproducibility

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

In [3]:
X = tf.placeholder(tf.float32,[None,4])
Y = tf.placeholder(tf.float32,[None,3])
nb_classes = 3

In [4]:
W = tf.Variable(tf.random_normal([4,nb_classes]),name="weight")
b = tf.Variable(tf.random_normal([nb_classes]),name='bias')

In [5]:
hypothesis = tf.nn.softmax(tf.matmul(X,W) + b)

In [6]:
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
# Y : one-hot encoded result 
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    print('--------------')

    # Testing & One-hot encoding
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a, 1)))

    print('--------------')

    b = sess.run(hypothesis, feed_dict={X: [[1, 3, 4, 3]]})
    print(b, sess.run(tf.argmax(b, 1)))

    print('--------------')

    c = sess.run(hypothesis, feed_dict={X: [[1, 1, 0, 1]]})
    print(c, sess.run(tf.argmax(c, 1)))

    print('--------------')

    all = sess.run(hypothesis, feed_dict={
                   X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, sess.run(tf.argmax(all, 1)))

0 5.04862
200 0.548425
400 0.432148
600 0.352466
800 0.276829
1000 0.232683
1200 0.210553
1400 0.192215
1600 0.176752
1800 0.163534
2000 0.152108
--------------
[[  1.38904783e-03   9.98601973e-01   9.06132664e-06]] [1]
--------------
[[ 0.93119192  0.06290215  0.00590591]] [0]
--------------
[[  1.27328148e-08   3.34113516e-04   9.99665856e-01]] [2]
--------------
[[  1.38904783e-03   9.98601973e-01   9.06133482e-06]
 [  9.31191921e-01   6.29021451e-02   5.90590853e-03]
 [  1.27328148e-08   3.34113516e-04   9.99665856e-01]] [1 0 2]


## Fancy Softmax Classifier

In [8]:
xy = np.loadtxt('DeepLearningZeroToAll/data-04-zoo.csv', delimiter= ',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

In [9]:
nb_classes = 7
X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None,1]) 
# 0~6 shape = (?,1)
Y_one_hot = tf.one_hot(Y, nb_classes)
# one hot shape = (?, 1, 7)
Y_one_hot = tf.reshape(Y_one_hot, [-1 ,nb_classes])
#shape = (?, 7)

In [10]:
W = tf.Variable(tf.random_normal([16, nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]),name = 'bias')

logits = tf.matmul(X,W) + b
hypothesis = tf.nn.softmax(logits)

cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels = Y_one_hot)

In [11]:
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [12]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={
                                 X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
                step, loss, acc))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

Step:     0	Loss: 4.685	Acc: 16.83%
Step:   100	Loss: 0.466	Acc: 86.14%
Step:   200	Loss: 0.325	Acc: 91.09%
Step:   300	Loss: 0.250	Acc: 93.07%
Step:   400	Loss: 0.202	Acc: 97.03%
Step:   500	Loss: 0.169	Acc: 98.02%
Step:   600	Loss: 0.146	Acc: 99.01%
Step:   700	Loss: 0.128	Acc: 99.01%
Step:   800	Loss: 0.114	Acc: 100.00%
Step:   900	Loss: 0.103	Acc: 100.00%
Step:  1000	Loss: 0.094	Acc: 100.00%
Step:  1100	Loss: 0.087	Acc: 100.00%
Step:  1200	Loss: 0.080	Acc: 100.00%
Step:  1300	Loss: 0.075	Acc: 100.00%
Step:  1400	Loss: 0.070	Acc: 100.00%
Step:  1500	Loss: 0.066	Acc: 100.00%
Step:  1600	Loss: 0.062	Acc: 100.00%
Step:  1700	Loss: 0.059	Acc: 100.00%
Step:  1800	Loss: 0.056	Acc: 100.00%
Step:  1900	Loss: 0.053	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3

## Batch Loader

In [13]:
print(x_data.shape)
x_data

(101, 16)


array([[ 1.,  0.,  0., ...,  0.,  0.,  1.],
       [ 1.,  0.,  0., ...,  1.,  0.,  1.],
       [ 0.,  0.,  1., ...,  1.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  1.,  0.,  1.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  1.,  0.,  0.]], dtype=float32)

In [18]:
print(y_data.shape)
y_data

(101, 1)


array([[ 0.],
       [ 0.],
       [ 3.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 3.],
       [ 3.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 3.],
       [ 6.],
       [ 6.],
       [ 6.],
       [ 1.],
       [ 0.],
       [ 3.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 5.],
       [ 4.],
       [ 4.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 5.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 3.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 3.],
       [ 5.],
       [ 5.],
       [ 1.],
       [ 5.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 6.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 5.],
       [ 4.],
       [ 6.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 3.],
       [ 3.],
       [ 2.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
      

In [15]:
filename_queue = tf.train.string_input_producer(
    ['DeepLearningZeroToAll/data-04-zoo.csv'],shuffle = False, name ="filename_queue")

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

record_defaults = [tf.constant([], dtype=tf.float32) for i in range(16)] + [tf.constant([], dtype=tf.int32)]
xy = tf.decode_csv(value, record_defaults = record_defaults, field_delim=',')

train_x_batch , train_y_batch = \
    tf.train.batch([xy[0:-1],xy[-1:]],batch_size = 10)

In [16]:
nb_classes = 7
X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None,1]) 
# 0~6 shape = (?,1)
Y_one_hot = tf.one_hot(Y, nb_classes)
# one hot shape = (?, 1, 7)
Y_one_hot = tf.reshape(Y_one_hot, [-1 ,nb_classes])
#shape = (?, 7)

W = tf.Variable(tf.random_normal([16, nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]),name = 'bias')

logits = tf.matmul(X,W) + b
hypothesis = tf.nn.softmax(logits)

cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels = Y_one_hot)

cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [17]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Start populating the filename queue.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    print("x-data")
    print(x_batch)
    print("y-data")
    print(y_batch)
    
    coord.request_stop()
    coord.join(threads)

x-data
[[ 1.  0.  0.  1.  0.  0.  1.  1.  1.  1.  0.  0.  4.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.  0.  1.  1.  1.  0.  0.  4.  1.  0.  1.]
 [ 0.  0.  1.  0.  0.  1.  1.  1.  1.  0.  0.  1.  0.  1.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  1.  1.  1.  1.  0.  0.  4.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.  1.  1.  1.  1.  0.  0.  4.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.  0.  1.  1.  1.  0.  0.  4.  1.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.  0.  1.  1.  1.  0.  0.  4.  1.  1.  1.]
 [ 0.  0.  1.  0.  0.  1.  0.  1.  1.  0.  0.  1.  0.  1.  1.  0.]
 [ 0.  0.  1.  0.  0.  1.  1.  1.  1.  0.  0.  1.  0.  1.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.  1.  1.  1.  0.  0.  4.  0.  1.  0.]]
y-data
[[0]
 [0]
 [3]
 [0]
 [0]
 [0]
 [0]
 [3]
 [3]
 [0]]


In [19]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # Start populating the filename queue.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    
    for step in range(2000):
        
        x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
        
        _, loss, acc = sess.run([optimizer, cost, accuracy], feed_dict={
                                 X: x_batch, Y: y_batch})
        if step % 100 == 0:
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
                step, loss, acc))

            
    coord.request_stop()
    coord.join(threads)
    
    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

Step:     0	Loss: 9.890	Acc: 0.00%
Step:   100	Loss: 0.468	Acc: 90.00%
Step:   200	Loss: 0.734	Acc: 70.00%
Step:   300	Loss: 0.784	Acc: 60.00%
Step:   400	Loss: 0.184	Acc: 100.00%
Step:   500	Loss: 0.238	Acc: 100.00%
Step:   600	Loss: 0.069	Acc: 100.00%
Step:   700	Loss: 0.029	Acc: 100.00%
Step:   800	Loss: 0.132	Acc: 100.00%
Step:   900	Loss: 0.104	Acc: 100.00%
Step:  1000	Loss: 0.021	Acc: 100.00%
Step:  1100	Loss: 0.067	Acc: 100.00%
Step:  1200	Loss: 0.181	Acc: 100.00%
Step:  1300	Loss: 0.234	Acc: 100.00%
Step:  1400	Loss: 0.039	Acc: 100.00%
Step:  1500	Loss: 0.104	Acc: 100.00%
Step:  1600	Loss: 0.033	Acc: 100.00%
Step:  1700	Loss: 0.019	Acc: 100.00%
Step:  1800	Loss: 0.048	Acc: 100.00%
Step:  1900	Loss: 0.042	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction

# decoded csv error 

\# 과 같은 주석이 csv 파일 위에 있을 경우, 지워주어야 한다.
한줄인 경우, skip-header를 활용하면 됨.
여러줄이면 어떻게할지 찾아볼것